In [2]:
import json
from pyspark import sql
from pyspark.sql import functions as sfunc
import pandas as pd
pd.set_option('display.max_columns', 50)

In [3]:
sc = sql.SparkSession.builder.appName('test')\
                            .master('local[2]')\
                            .config("spark.sql.warehouse.dir", "hdfs:/localhost:9000/user/hive/warehouse")\
                            .enableHiveSupport()\
                            .getOrCreate()

24/03/02 02:04:42 WARN Utils: Your hostname, Ajay resolves to a loopback address: 127.0.0.1; using 172.25.184.63 instead (on interface eth0)
24/03/02 02:04:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 02:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc.sparkContext.getConf().getAll()

[('spark.driver.port', '33189'),
 ('spark.app.name', 'test'),
 ('spark.app.id', 'local-1709363084700'),
 ('spark.sql.warehouse.dir', 'hdfs:/localhost:9000/user/hive/warehouse'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5

In [5]:
df = sc.read.json("file:/home/ajay/yelp_dataset/yelp_academic_dataset_user.json", dropFieldIfAllNull = True)

In [7]:
sc.sparkContext.getConf().getAll()

[('spark.driver.port', '33189'),
 ('spark.app.name', 'test'),
 ('spark.app.id', 'local-1709363084700'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir',
  'hdfs://localhost:9000/localhost:9000/user/hive/warehouse'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss

In [7]:
df = df.withColumns({'friends' : sfunc.array_distinct(sfunc.split('friends', ", ")),
                'elite_years' : sfunc.split('elite', ', ')}).drop('elite').withColumns({
                'primary_reach' : sfunc.size(sfunc.col('friends')),
                'number_of_elite_years' : sfunc.size(sfunc.col('elite_years')),
                'yelping_since' : sfunc.to_timestamp(sfunc.col('yelping_since'))
})

In [8]:
sc.sparkContext.getConf().getAll()

[('spark.app.name', 'test'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '35799'),
 ('spark.app.startTime', '1709361883853'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.sql.wa

In [9]:
secondary_reach = df.select(sfunc.explode('friends').alias('friend_id'), 'user_id'
                           ).groupBy('user_id').agg(
                                                    sfunc.size(
                                                        sfunc.array_distinct(
                                                            sfunc.array_agg(sfunc.col('friend_id'))
                                                        )
                                                    ).alias("secondary_reach")
                                                )
df = df.join(secondary_reach, on = 'user_id', how = 'left').drop('friends')

In [10]:
# sc.sql("CREATE DATABASE IF NOT EXISTS yelp")
df.write.saveAsTable("default.user_details")

24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.metastore.wm.default.pool.size does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.llap.output.format.arrow does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.arrow.root.allocator.limit does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.vectorized.use.checked.expressions does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.tez.dynamic.semijoin.reduction.for.mapjoin does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.vectorized.complex.types.enabled does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.server2.wm.worker.threads does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.repl.partitions.dump.

24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.metastore.wm.default.pool.size does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.llap.output.format.arrow does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.arrow.root.allocator.limit does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.vectorized.use.checked.expressions does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.tez.dynamic.semijoin.reduction.for.mapjoin does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.vectorized.complex.types.enabled does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.server2.wm.worker.threads does not exist
24/03/02 01:47:42 WARN HiveConf: HiveConf of name hive.repl.partitions.dump.

24/03/02 01:47:43 WARN DriverDataSource: Registered driver with driverClassName=org.apache.derby.jdbc.EmbeddedDriver was not found, trying direct instantiation.
24/03/02 01:47:44 WARN DriverDataSource: Registered driver with driverClassName=org.apache.derby.jdbc.EmbeddedDriver was not found, trying direct instantiation.
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.metastore.wm.default.pool.size does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.llap.task.scheduler.preempt.independent does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.llap.output.format.arrow does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.tez.llap.min.reducer.per.executor does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.arrow.root.allocator.limit does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.vectorized.use.checked.expressions does not exist
24/03/02 01:47:44 WARN HiveConf: HiveConf of name hive.tez.dynamic

24/03/02 01:47:46 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/03/02 01:47:46 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ajay@127.0.1.1


IllegalArgumentException: Pathname /localhost:9000/user/hive/warehouse/user_details from hdfs://localhost:9000/localhost:9000/user/hive/warehouse/user_details is not a valid DFS filename.

In [ ]:
sc.sparkContext.getConf().get("spark.sql.catalog.spark_catalog")